<a href="https://colab.research.google.com/github/ChrisTheDragon/Minha-Rede-Neural/blob/main/Aprendizado_com_TFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Como usar Tensorflow-federated para construir uma rede com aprendizado federado**

## O que é `Tensorflow-federated`

A biblioteca TensorFlow Federated (TFF) é uma extensão do TensorFlow, que é um dos frameworks de aprendizado de máquina mais populares atualmente. O TFF foi projetado para permitir a construção e treinamento de modelos de aprendizado de máquina em um contexto de aprendizado federado. No aprendizado federado, em vez de reunir todos os dados em um local centralizado, o treinamento do modelo ocorre em dispositivos ou sistemas distribuídos, preservando a privacidade dos dados locais.

## Importações

In [35]:
#@title
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

## Preparando os dados de entrada
A aprendizagem federada requer um conjunto de dados federados, ou seja, uma coleção de dados de vários usuários.

A fim de facilitar a experimentação, o repositório TFF tem alguns conjuntos de dados que podem ser usados livremente como teste, incluindo uma versão federado de MNIST que contém uma versão do conjunto de dados NIST originais. Que são digitos de 0-9 escritos a mão por escritores diferentes. Cada escritor será tratado como um cliente diferente.


Para alimentar o conjunto de dados em nosso modelo, achatar os dados, e converter cada exemplo em uma tupla da forma (flattened_image_vector, label).

In [37]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

A função preprocess recebe um conjunto de dados, divide-o em lotes de tamanho fixo `BATCH_SIZE` e remodela os tensores de pixels e rótulos para uma forma adequada. Isso permite que os dados sejam processados em lotes durante o treinamento da rede neural no contexto do aprendizado federado.

In [38]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

O código acima cria uma lista chamada `federated_train_data`, que contém os conjuntos de dados pré-processados para cada cliente (dispositivo) envolvido no aprendizado federado. 

## Construindo o Modelo
Este modelo (implementado através `tf.keras` ) tem uma única camada oculta, seguindo-se uma camada Softmax.